In [3]:
import pandas as pd

df = pd.read_csv('C:/Users/BNC/Documents/ITC-Internship/LLM/LLM-Model/visal/dataset_khmer.csv', encoding='utf-8')
print(df.columns)
df

Index(['Action Sentence                                                                                       ', 'Formatted Output'], dtype='object')


,Action Sentence,Formatted Output
0,បកក្រោយ ១.៦៨ ម៉ែត្រ នៅល្បឿន ៩.៥៣ ម៉ែត្រ/វិនាទី...,"{'action': 'move', 'params': {'linear_speed': ..."
1,"ទៅមុខ ១០.១១ ម៉ែត្រ នៅល្បឿន ៥.៣៨ ម៉ែត្រ/វិនាទី,...","{'action': 'move', 'params': {'linear_speed': ..."
2,"ទៅមុខ ១២.២១ ម៉ែត្រ នៅល្បឿន ៤.៥ ម៉ែត្រ/វិនាទី, ...","{'action': 'move', 'params': {'linear_speed': ..."
3,បកក្រោយ ១៤.៣ ម៉ែត្រ ដោយល្បឿន ៩.៦៥ ម៉ែត្រក្នុង១...,"{'action': 'move', 'params': {'linear_speed': ..."
4,ទៅមុខ ៨.៤៣ ម៉ែត្រ ក្នុងល្បឿន ៥.៨ ម៉ែត្រ/វិនាទី...,"{'action': 'move', 'params': {'linear_speed': ..."
...,...,...
495,បកក្រោយ 17.52 ម៉ែត្រ នៅល្បឿន 7.73 ម៉ែត្រ/វិនាទ...,"{'action': 'move', 'params': {'linear_speed': ..."
496,ត្រឡប់ទៅក្រោយ 2.78 ម៉ែត្រ នៅល្បឿន 6.84 ម៉ែត្រ/...,"{'action': 'move', 'params': {'linear_speed': ..."
497,"ទៅមុខ 6.86 ម៉ែត្រ នៅល្បឿន 4.4 ម៉ែត្រ/វិនាទី, ប...","{'action': 'move', 'params': {'linear_speed': ..."
498,"ទៅមុខ 1.07 ម៉ែត្រ នៅល្បឿន 7.13 ម៉ែត្រ/វិនាទី, ...","{'action': 'move', 'params': {'linear_speed': ..."


## all-MiniLM-L6-v2

In [4]:
def preprocess_data(data):
    inputs = [item['question'] for item in data]  # Use correct column name
    outputs = [item['answer'] for item in data]  # Use correct column name
    return inputs, outputs
print(input)

<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x00000206E964A7C0>>


In [5]:
def preprocess_data(data):
    if 'input' not in data[0] or 'output' not in data[0]:
        raise ValueError("Dataset is missing 'input' or 'output' columns. Available columns: " + str(data[0].keys()))
    
    inputs = [item['input'] for item in data]
    outputs = [item['output'] for item in data]
    return inputs, outputs
print(input)


<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x00000206E964A7C0>>


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Step 1: Load and Clean CSV Dataset
def load_dataset(filepath):
    df = pd.read_csv(filepath, encoding='utf-8')
    df.columns = df.columns.str.strip()  # Remove trailing spaces
    print("Fixed Columns:", df.columns)  # Debugging
    return df.to_dict(orient='records')  # Convert to list of dictionaries

# Step 2: Preprocess Data
def preprocess_data(data):
    if 'input' not in data[0] or 'output' not in data[0]:
        raise ValueError(f"Dataset column mismatch! Found: {list(data[0].keys())}")

    inputs = [item['input'] for item in data]
    outputs = [item['output'] for item in data]
    return inputs, outputs

# Step 3: Sentence Embeddings
def build_embeddings(inputs):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(inputs, convert_to_tensor=True)
    return model, embeddings

# Step 4: Get chatbot response
def get_response(user_input, model, embeddings, inputs, outputs):
    user_input_embedding = model.encode(user_input, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(user_input_embedding, embeddings)
    best_match_idx = similarities.argmax()
    return outputs[best_match_idx]

# Main chatbot function
def chatbot(filepath):
    data = load_dataset(filepath)
    inputs, outputs = preprocess_data(data)  # FIX: Extract inputs and outputs
    model, embeddings = build_embeddings(inputs)  # Now inputs is defined

    print("Chatbot Ready! Type 'exit' to quit.\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye!")
            break
        response = get_response(user_input, model, embeddings, inputs, outputs)
        print(f"Chatbot: {response}")
        print(f"user_inputs: {user_input}")

# Run chatbot
#chatbot('C:/Users/BNC/Documents/ITC-Internship/LLM/LLM-Model/visal/dataset_khmer.csv')
chatbot('C:/Users/BNC/Documents/ITC-Internship/LLM\LLM-Model/visal/Navigatintion_robot_datasets_eng.csv')

Fixed Columns: Index(['input', 'output'], dtype='object')


## facebook/m2m100_1.2B

In [ ]:
import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import pandas as pd

# Load M2M100 Model
model_name = "facebook/m2m100_1.2B"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)

# Load and preprocess dataset
def load_dataset(filepath):
    df = pd.read_csv(filepath, encoding='utf-8')
    df.columns = df.columns.str.strip()
    return df.to_dict(orient='records')

def preprocess_data(data):
    if 'Action Sentence' not in data[0] or 'Formatted Output' not in data[0]:
        raise ValueError(f"Dataset column mismatch! Found: {list(data[0].keys())}")

    inputs = [item['Action Sentence'] for item in data]
    outputs = [item['Formatted Output'] for item in data]
    return inputs, outputs

# Function to translate text
def translate_text(text, src_lang="km", tgt_lang="en"):  # Use 'km' for Khmer
    tokenizer.src_lang = src_lang
    encoded_text = tokenizer(text, return_tensors="pt")

    with torch.no_grad():
        generated_tokens = model.generate(**encoded_text, forced_bos_token_id=tokenizer.get_lang_id(tgt_lang))

    return tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

# Main chatbot function
def chatbot(filepath):
    data = load_dataset(filepath)
    inputs, outputs = preprocess_data(data)
    
    print("M2M100 Chatbot is ready! Type 'exit' to quit.\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye!")
            break
        
        # Translate Khmer input to English
        translated_input = translate_text(user_input, src_lang="km", tgt_lang="en")  # Fixed language code
        print(f"Translated Input: {translated_input}")

        # Find the best-matching response (simple keyword matching for now)
        best_response = next((output for inp, output in zip(inputs, outputs) if inp.lower() in translated_input.lower()), "Sorry, I don't understand.")

        # Translate response back to Khmer
        translated_response = translate_text(best_response, src_lang="en", tgt_lang="km")  # Fixed language code
        print(f"user: {user_input}")
        print(f"Chatbot: {translated_response}")

# Run chatbot
chatbot('C:/Users/BNC/Documents/ITC-Internship/LLM/LLM-Model/visal/dataset_khmer.csv')



In [ ]:
import pandas as pd
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Define test dataset (Khmer commands and expected JSON output)
test_data = [
    ("ត្រឡប់ទៅក្រោយ 5.24 ម៉ែត្រ នៅល្បឿន 1.33 ម៉ែត្រ/វិនាទី, បន្ទាប់មក បង្វែស្ដាំ 160 ដឺក្រេ។",
     "{'action': 'move', 'params': {'linear_speed': 1.33, 'distance': 5.24, 'is_forward': False, 'unit': 'meter'}}\n"
     "{'action': 'rotate', 'params': {'angle': 160, 'is_clockwise': True, 'unit': 'degree'}}"),
    
    ("ទៅមុខ 4.44 ម៉ែត្រ នៅល្បឿន 7.68 ម៉ែត្រ/វិនាទី, បន្ទាប់មក បង្វែស្ដាំ 228 ដឺក្រេ។",
     "{'action': 'move', 'params': {'linear_speed': 7.68, 'distance': 4.44, 'is_forward': True, 'unit': 'meter'}}\n"
     "{'action': 'rotate', 'params': {'angle': 228, 'is_clockwise': True, 'unit': 'degree'}}"),
    
    ("ត្រឡប់ទៅក្រោយ 1.11 ម៉ែត្រ នៅល្បឿន 4.04 ម៉ែត្រ/វិនាទី, បន្ទាប់មក បត់ស្ដាំ 42 ដឺក្រេ។",
     "{'action': 'move', 'params': {'linear_speed': 4.04, 'distance': 1.11, 'is_forward': False, 'unit': 'meter'}}\n"
     "{'action': 'rotate', 'params': {'angle': 42, 'is_clockwise': True, 'unit': 'degree'}}"),
]

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"

# Model name for testing (only Flan-T5 Large)
model_name = "google/flan-t5-large"

# Store results
results = []

try:
    # Load model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

    # Run inference on test dataset
    for input_text, expected_output in test_data:
        # Tokenize input
        inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(device)
        
        # Generate output
        with torch.no_grad():
            outputs = model.generate(**inputs, max_length=256)
        
        # Decode generated text
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Store results
        results.append({
            "Model": model_name,
            "Input": input_text,
            "Expected Output": expected_output,
            "Generated Output": generated_text
        })

except Exception as e:
    results.append({
        "Model": model_name,
        "Input": "Error",
        "Expected Output": "Error",
        "Generated Output": f"Model failed to load: {str(e)}"
    })

#  Convert results to a DataFrame and display
df_results = pd.DataFrame(results)

In [ ]:
import pandas as pd
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

#  Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "google/flan-t5-large"

try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    print("\n Chatbot is ready! Type 'exit' to quit.\n")
except Exception as e:
    print(f" Model loading failed: {str(e)}")
    exit()

#  Define test dataset (Khmer commands and expected JSON output)
test_data = [
    ("ត្រឡប់ទៅក្រោយ 5.24 ម៉ែត្រ នៅល្បឿន 1.33 ម៉ែត្រ/វិនាទី, បន្ទាប់មក បង្វែស្ដាំ 160 ដឺក្រេ។",
     "{'action': 'move', 'params': {'linear_speed': 1.33, 'distance': 5.24, 'is_forward': False, 'unit': 'meter'}}\n"
     "{'action': 'rotate', 'params': {'angle': 160, 'is_clockwise': True, 'unit': 'degree'}}"),
    
    ("ទៅមុខ 4.44 ម៉ែត្រ នៅល្បឿន 7.68 ម៉ែត្រ/វិនាទី, បន្ទាប់មក បង្វែស្ដាំ 228 ដឺក្រេ។",
     "{'action': 'move', 'params': {'linear_speed': 7.68, 'distance': 4.44, 'is_forward': True, 'unit': 'meter'}}\n"
     "{'action': 'rotate', 'params': {'angle': 228, 'is_clockwise': True, 'unit': 'degree'}}"),
    
    ("ត្រឡប់ទៅក្រោយ 1.11 ម៉ែត្រ នៅល្បឿន 4.04 ម៉ែត្រ/វិនាទី, បន្ទាប់មក បត់ស្ដាំ 42 ដឺក្រេ។",
     "{'action': 'move', 'params': {'linear_speed': 4.04, 'distance': 1.11, 'is_forward': False, 'unit': 'meter'}}\n"
     "{'action': 'rotate', 'params': {'angle': 42, 'is_clockwise': True, 'unit': 'degree'}}"),
]

#  Function to run chatbot inference
def chatbot():
    while True:
        # Get user input
        user_input = input("You: ")
        
        # Exit condition
        if user_input.lower() == "exit":
            print("\nChatbot: Goodbye! 👋")
            break
        
        # Tokenize input
        inputs = tokenizer(user_input, return_tensors="pt", padding=True, truncation=True).to(device)
        
        # Generate response
        with torch.no_grad():
            outputs = model.generate(**inputs, max_length=256)
        
        # Decode generated response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Display chatbot response
        print(f"Chatbot: {response}\n")

# Function to run model on test dataset
def run_test():
    results = []
    
    for input_text, expected_output in test_data:
        # Tokenize input
        inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(device)
        
        # Generate output
        with torch.no_grad():
            outputs = model.generate(**inputs, max_length=256)
        
        # Decode generated text
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Store results
        results.append({
            "Model": model_name,
            "Input": input_text,
            "Expected Output": expected_output,
            "Generated Output": generated_text
        })

    #  Convert results to a DataFrame and display
    df_results = pd.DataFrame(results)
    print("\n🔹 Model Test Results 🔹")
    print(df_results)

#  Ask user whether to test or run chatbot
if __name__ == "__main__":
    choice = input("Type 'test' to run model test or 'chat' to start chatbot: ").strip().lower()
    
    if choice == "test":
        run_test()
    elif choice == "chat":
        chatbot()
    else:
        print(" Invalid choice. Please restart the script and choose 'test' or 'chat'.")


In [ ]:
!pip install bitsandbytes

   ---------------------------------------- 0.0/69.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/69.1 MB 330.3 kB/s eta 0:03:30
   ---------------------------------------- 0.1/69.1 MB 1.4 MB/s eta 0:00:48
   ---------------------------------------- 0.5/69.1 MB 4.1 MB/s eta 0:00:17
    --------------------------------------- 0.9/69.1 MB 6.2 MB/s eta 0:00:11
   - -------------------------------------- 2.2/69.1 MB 9.8 MB/s eta 0:00:07
   - -------------------------------------- 3.2/69.1 MB 12.0 MB/s eta 0:00:06
   -- ------------------------------------- 3.5/69.1 MB 13.1 MB/s eta 0:00:06
   -- ------------------------------------- 4.9/69.1 MB 14.3 MB/s eta 0:00:05
   --- ------------------------------------ 5.4/69.1 MB 13.7 MB/s eta 0:00:05
   --- ------------------------------------ 6.4/69.1 MB 14.5 MB/s eta 0:00:05
   ---- ----------------------------------- 7.0/69.1 MB 13.9 MB/s eta 0:00:05
   ---- ----------------------------------- 7.5/69.1 MB 14.1 MB/s eta 0:00:

### "SeaLLMs/SeaLLMs-v3-7B-Chat"

In [ ]:
import torch
import pandas as pd
import json
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import Trainer

# Configurations
MODEL_NAME = "SeaLLMs/SeaLLMs-v3-7B-Chat"
CSV_FILE = r"C:\Users\BNC\Documents\ITC-Internship\LLM\LLM-Model\visal\cleaned_dataset_Khmer.csv"
BATCH_SIZE = 1  # Optimized for low VRAM
GRADIENT_ACCUMULATION_STEPS = 8
EPOCHS = 3  
LORA_R = 8  
LORA_ALPHA = 16
LORA_DROPOUT = 0.1

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, padding_side="right", truncation_side="right")


In [ ]:

# Load dataset from CSV
def load_data(csv_file):
    df = pd.read_csv(csv_file)

    # Ensure correct column names
    if "khmer_input" not in df.columns or "robotic_command" not in df.columns:
        raise ValueError("CSV file must have 'khmer_input' and 'robotic_command' columns.")

    # Clean dataset: Remove malformed JSON entries
    cleaned_data = []
    for _, row in df.iterrows():
        try:
            json.loads(row["robotic_command"].replace("'", "\""))  # Validate JSON format
            cleaned_data.append(row)
        except json.JSONDecodeError:
            continue  # Skip invalid rows

    # Convert back to DataFrame
    clean_df = pd.DataFrame(cleaned_data)
    dataset = Dataset.from_pandas(clean_df)

    return dataset

dataset = load_data(CSV_FILE)
print(dataset)

Dataset({
    features: ['khmer_input', 'robotic_command', '__index_level_0__'],
    num_rows: 1198
})


In [ ]:
import torch
import os
import pandas as pd
import json
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import Trainer

# ✅ **Configurations**
MODEL_NAME = "SeaLLMs/SeaLLMs-v3-7B-Chat"
CSV_FILE = r"C:\Users\BNC\Documents\ITC-Internship\LLM\LLM-Model\visal\cleaned_dataset_Khmer.csv"
BATCH_SIZE = 1  # Optimized for low VRAM
GRADIENT_ACCUMULATION_STEPS = 8
EPOCHS = 3  
LORA_R = 8  
LORA_ALPHA = 16
LORA_DROPOUT = 0.1

# ✅ **Ensure Offloading Directory Exists**
os.makedirs("./offload", exist_ok=True)

# ✅ **Load tokenizer**
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, padding_side="right", truncation_side="right")

# ✅ **Load and Clean Dataset**
def load_data(csv_file):
    df = pd.read_csv(csv_file)

    # Ensure correct column names
    if "khmer_input" not in df.columns or "robotic_command" not in df.columns:
        raise ValueError("CSV file must have 'khmer_input' and 'robotic_command' columns.")

    # Clean dataset: Remove malformed JSON entries
    cleaned_data = []
    for _, row in df.iterrows():
        try:
            json.loads(row["robotic_command"].replace("'", "\""))  # Validate JSON format
            cleaned_data.append(row)
        except json.JSONDecodeError:
            continue  # Skip invalid rows

    # Convert back to DataFrame
    clean_df = pd.DataFrame(cleaned_data)
    dataset = Dataset.from_pandas(clean_df)

    return dataset

dataset = load_data(CSV_FILE)

# ✅ **Tokenization**
def tokenize_function(example):
    input_text = f"<User>: {example['khmer_input']}\n<Assistant>: "
    output_text = example["robotic_command"]

    # Tokenize both input and output properly
    inputs = tokenizer(input_text, padding="max_length", max_length=256, truncation=True)
    outputs = tokenizer(output_text, padding="max_length", max_length=256, truncation=True)

    # Ensure labels are formatted correctly
    inputs["labels"] = outputs["input_ids"]

    return inputs

# ✅ **Apply Tokenization**
dataset = dataset.map(tokenize_function, batched=False)  # **Set batched=False**
dataset = dataset.train_test_split(test_size=0.1)

# ✅ **Load Model Fully on CPU (Fixes VRAM Crash)**
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="cpu",  # ✅ Fully Offload Model to CPU
)

# ✅ **Ensure LoRA is Applied Before Training**
model = prepare_model_for_kbit_training(model)  # ✅ Fixes compatibility issues

lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# ✅ **Training arguments (Optimized for CPU)**
training_args = TrainingArguments(
    output_dir="./fine_tuned_seallm",
    evaluation_strategy="epoch",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=1e-4,
    num_train_epochs=EPOCHS,
    save_total_limit=2,
    report_to="none"
)

# ✅ **Data collator**
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)

# ✅ **Trainer (CPU Only Training)**
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

# ✅ **Train Model (Runs on CPU)**
trainer.train()

# ✅ **Save Model**
trainer.save_model("./fine_tuned_seallm")
tokenizer.save_pretrained("./fine_tuned_seallm")

print("✅ Training complete! Model saved at './fine_tuned_seallm'")


c:\Users\BNC\anaconda3\envs\airsim_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [01:04<00:00, 16.24s/it]
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
c:\Users\BNC\anaconda3\envs\airsim_env\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\BNC\AppData\Local\Temp\ipykernel_11080\3776360960.py:108: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  0%|          | 0/402 [00:00<?, ?it/s]

In [ ]:
!pip install bitsandbytes --no-cache-dir --force-reinstall

   ---------------------------------------- 0.0/69.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/69.1 MB 3.3 MB/s eta 0:00:21
   ---------------------------------------- 0.2/69.1 MB 2.5 MB/s eta 0:00:28
   ---------------------------------------- 0.3/69.1 MB 2.2 MB/s eta 0:00:32
   ---------------------------------------- 0.4/69.1 MB 2.3 MB/s eta 0:00:31
   ---------------------------------------- 0.4/69.1 MB 2.3 MB/s eta 0:00:31
   ---------------------------------------- 0.5/69.1 MB 1.5 MB/s eta 0:00:47
   ---------------------------------------- 0.6/69.1 MB 1.8 MB/s eta 0:00:40
   ---------------------------------------- 0.8/69.1 MB 2.0 MB/s eta 0:00:34
    --------------------------------------- 0.9/69.1 MB 1.9 MB/s eta 0:00:36
    --------------------------------------- 1.0/69.1 MB 2.0 MB/s eta 0:00:34
    --------------------------------------- 1.0/69.1 MB 1.9 MB/s eta 0:00:35
    --------------------------------------- 1.0/69.1 MB 1.9 MB/s eta 0:00:35
    --

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.1.0 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have fsspec 2025.2.0 which is incompatible.
tensorflow-intel 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 1.24.4 which is incompatible.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_PATH = "./fine_tuned_seallm"

# Load trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, torch_dtype=torch.float16, device_map="auto")

# Test input
test_sentence = "តោះទៅមុខ ៥ ម៉ែត្រ"

# Format prompt
messages = [
    {"role": "system", "content": "You are a helpful assistant that converts Khmer instructions to robotics commands."},
    {"role": "user", "content": test_sentence}
]
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Encode input
model_inputs = tokenizer([text], return_tensors="pt").to("cuda")

# Generate output
generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=100, do_sample=True)
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(f"Input: {test_sentence}")
print(f"Output: {response}")


Test the model: